# Example how to query multiple items using paging

In [1]:
import sys
from sgqlc.operation import Operation
from pdp_schema import pdp_schema as schema
from pdp_graphql_client_python import client

In [2]:
! echo "${PDP_API}"

https://graphql-api.pdp.int.srgssr.ch/graphql


In [3]:
# show available queries
# *Page queries allow for paging (https://graphql.org/learn/pagination/)
# first=[number of queries to retrieve, e.g. *first* 10 queries]
# after=[id or cursor position from which to start]
schema.Query

type Query {
  faroProgramPage(first: Int!, after: String): FaroProgramPage!
  faroPrograms(ids: [String!]!): [FaroProgram]!
  randomFaroPrograms: [FaroProgram!]!
  faroItemPage(first: Int!, after: String): FaroItemPage!
  faroItems(ids: [String!]!): [FaroItem]!
  faroItemsByPlayUrn(urns: [String!]!): [FaroItem]!
  faroItemsByPlayUrnGraphDb(urns: [String!]!): [FaroItem]!
  randomFaroItems: [FaroItem!]!
}

In [4]:
from sgqlc.types import Variable, non_null
# retrieve query operations
op = Operation(schema.Query, name='PagingQuery', variables={'first':non_null(int), 'after':str})

In [5]:
type(op)

sgqlc.operation.Operation

In [6]:
# return current query (empty)
op

query PagingQuery($first: Int!, $after: String) {
}

In [7]:
# select query type and configure with argument
faro = op.faro_program_page(first=Variable('first'), after=Variable('after'))

In [8]:
# return current query (empty)
op

query PagingQuery($first: Int!, $after: String) {
  faroProgramPage(first: $first, after: $after) {
    edges {
      id
      bu
      mediaType
      episodeIds
      sTit
      sDatStart
      sGef
      pTit
      department
      workgroup
      seriesNr
    }
    cursor
  }
}

In [9]:
# select fields to be retured
faro.edges.media_type()
faro.edges.s_dat_start()
faro.edges.s_gef()
faro.edges.s_tit()
faro.edges.series_nr()
faro.cursor()

cursor

In [10]:
# return current query
op

query PagingQuery($first: Int!, $after: String) {
  faroProgramPage(first: $first, after: $after) {
    edges {
      mediaType
      sDatStart
      sGef
      sTit
      seriesNr
    }
    cursor
  }
}

In [11]:
# constants
page_size = 10
page_requests = 2

In [39]:
data = client.run_query(op, {'first':page_size})

results = (op + data).faro_program_page

if not results.edges:
    print('No items!', file=sys.stderr)
    raise SystemExit()

In [13]:
# todo tqdm
for i in range(page_requests):
    # query page
    data = client.run_query(op, {'first':page_size, 'after':results.cursor})
    
    # convert page to object oriented results
    page = (op + data).faro_program_page

    # extract new edges and update cursor
    results.edges += page.edges
    results.cursor = page.cursor

Downloaded 10 pages after: MDAwMDYyNWYtMjkwMi00ZTVhLWFlYjMtNmVjMTZjNjZmNjI2
Downloaded 10 pages after: MDAwMGRjZTAtMDFhZC00ZWRmLThmY2MtZDhhOTE5NzM5MTdi


In [14]:
# display data converted as objects
results

FaroProgramPage(edges=[FaroProgram(media_type='audio', s_dat_start='1994-06-26', s_gef='Doppelpunkt', s_tit=None, series_nr=0), FaroProgram(media_type='audio', s_dat_start='2012-11-26', s_gef=None, s_tit=None, series_nr=None), FaroProgram(media_type='audio', s_dat_start='2006-11-21', s_gef='Oz', s_tit='Contribuziun', series_nr=None), FaroProgram(media_type='audio', s_dat_start='2012-09-28', s_gef=None, s_tit=None, series_nr=None), FaroProgram(media_type=None, s_dat_start=None, s_gef=None, s_tit=None, series_nr=0), FaroProgram(media_type='audio', s_dat_start='2018-04-21', s_gef=None, s_tit=None, series_nr=0), FaroProgram(media_type='audio', s_dat_start='1979-03-20', s_gef='Wir lesen vor', s_tit='Fortsetzung folgt: Eine Krähe war mit mir\n', series_nr=10), FaroProgram(media_type='audio', s_dat_start='2009-06-18', s_gef=None, s_tit=None, series_nr=None), FaroProgram(media_type='audio', s_dat_start='2005-04-20', s_gef='Kulturtipp', s_tit=None, series_nr=None), FaroProgram(media_type='audio

In [15]:
from collections import Counter
from datetime import date

# count year occurrences
Counter(date.fromisoformat(x.s_dat_start).year for x in results.edges if x.s_dat_start)

Counter({1994: 1,
         2012: 4,
         2006: 1,
         2018: 3,
         1979: 1,
         2009: 1,
         2005: 1,
         2002: 1,
         2008: 2,
         2011: 1,
         1977: 1,
         2017: 3,
         2015: 1,
         2010: 2,
         1940: 1,
         2016: 1,
         2003: 1,
         2000: 1,
         2007: 1})

In [16]:
import pandas as pd

In [17]:
dir(results.edges[0])

['_ContainerTypeMeta__fields',
 '_ContainerType__get_type_for_selection',
 '_ContainerType__populate_field_data',
 '_ContainerType__populate_fields',
 '_ContainerType__populate_fields_from_selection_list',
 '__bytes__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__field_names__',
 '__fields_cache__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__interfaces__',
 '__iter__',
 '__json_data__',
 '__json_dump_args__',
 '__kind__',
 '__le__',
 '__len__',
 '__lt__',
 '__meta_fields__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__schema__',
 '__selection_list__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__to_json_value__',
 '__weakref__',
 'media_type',
 's_dat_start',
 's_gef',
 's_tit',
 'series_nr']

In [18]:
results.edges[0].__field_names__

('id',
 'bu',
 'media_type',
 'episode_ids',
 's_tit',
 's_dat_start',
 's_gef',
 'p_tit',
 'department',
 'workgroup',
 'series_nr',
 'moderators',
 'item_page',
 'play_episodes')

In [19]:
{key: value for key, value in results.edges[0].__dict__.items() if not key.startswith("__")}

{'media_type': 'audio',
 's_dat_start': '1994-06-26',
 's_gef': 'Doppelpunkt',
 's_tit': None,
 'series_nr': 0}

In [24]:
a = results.edges[0]

In [25]:
dir(a)

['_ContainerTypeMeta__fields',
 '_ContainerType__get_type_for_selection',
 '_ContainerType__populate_field_data',
 '_ContainerType__populate_fields',
 '_ContainerType__populate_fields_from_selection_list',
 '__bytes__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__field_names__',
 '__fields_cache__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__interfaces__',
 '__iter__',
 '__json_data__',
 '__json_dump_args__',
 '__kind__',
 '__le__',
 '__len__',
 '__lt__',
 '__meta_fields__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__schema__',
 '__selection_list__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__to_json_value__',
 '__weakref__',
 'media_type',
 's_dat_start',
 's_gef',
 's_tit',
 'series_nr']

In [30]:
a.__fields_cache__

OrderedDict([('media_type', mediaType: String),
             ('s_dat_start', sDatStart: String),
             ('s_gef', sGef: String),
             ('s_tit', sTit: String),
             ('series_nr', seriesNr: Int)])

In [34]:
a.__json_dump_args__

{'sort_keys': True, 'separators': (',', ':')}

In [35]:
data['data']['faroProgramPage']['edges']

[{'sGef': 'Telesguard',
  'sTit': None,
  'mediaType': 'video',
  'sDatStart': '2010-12-27',
  'seriesNr': 0},
 {'sGef': None,
  'sTit': None,
  'mediaType': 'video',
  'sDatStart': None,
  'seriesNr': 0},
 {'sGef': 'Espresso',
  'sTit': None,
  'mediaType': 'audio',
  'sDatStart': '2016-07-13',
  'seriesNr': 0},
 {'sGef': None,
  'sTit': None,
  'mediaType': 'audio',
  'sDatStart': '2012-01-09',
  'seriesNr': None},
 {'sGef': 'Marella',
  'sTit': None,
  'mediaType': 'audio',
  'sDatStart': '2018-02-18',
  'seriesNr': 0},
 {'sGef': 'Oops!',
  'sTit': None,
  'mediaType': 'video',
  'sDatStart': '2003-03-12',
  'seriesNr': None},
 {'sGef': None,
  'sTit': None,
  'mediaType': 'audio',
  'sDatStart': '2008-11-13',
  'seriesNr': None},
 {'sGef': None,
  'sTit': None,
  'mediaType': 'audio',
  'sDatStart': '2017-10-17',
  'seriesNr': 0},
 {'sGef': 'Arena',
  'sTit': None,
  'mediaType': 'video',
  'sDatStart': '2000-10-13',
  'seriesNr': 0},
 {'sGef': 'Tubii',
  'sTit': None,
  'mediaType

In [37]:
df = pd.DataFrame([x.__json_data__ for x in results.edges])

In [38]:
df

,sGef,sTit,mediaType,sDatStart,seriesNr
0,Doppelpunkt,None,audio,1994-06-26,0.0
1,None,None,audio,2012-11-26,NaN
2,Oz,Contribuziun,audio,2006-11-21,NaN
3,None,None,audio,2012-09-28,NaN
4,None,None,None,None,0.0
5,None,None,audio,2018-04-21,0.0
6,Wir lesen vor,Fortsetzung folgt: Eine Krähe war mit mir\n,audio,1979-03-20,10.0
7,None,None,audio,2009-06-18,NaN
8,Kulturtipp,None,audio,2005-04-20,NaN
9,Regionaljournal Zürich Schaffhausen,None,audio,2002-01-28,NaN
